In [ ]:
import cv2
import numpy as np
from onnx import numpy_helper
import os, statistics
import onnx
import onnxruntime as rt
from PIL import Image
from matplotlib.pyplot import imshow, imsave
import matplotlib.pyplot as plt
from scipy import special
import colorsys
import random
import time
import argparse
from pycocotools.coco import COCO
from tqdm import tqdm

In [ ]:
# this function is from yolo3.utils.letterbox_image
def letterbox_image(image, size):
    '''resize image with unchanged aspect ratio using padding'''
    iw, ih = image.size
    w, h = size
    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)

    image = image.resize((nw,nh), Image.BICUBIC)
    new_image = Image.new('RGB', size, (128,128,128))
    new_image.paste(image, ((w-nw)//2, (h-nh)//2))
    return new_image

def preprocess(img):
    model_image_size = (416, 416)
    boxed_image = letterbox_image(img, tuple(reversed(model_image_size)))
    image_data = np.array(boxed_image, dtype='float32')
    image_data /= 255.
    image_data = np.transpose(image_data, [2, 0, 1])
    image_data = np.expand_dims(image_data, 0)
    return image_data

def categoryID2name(coco_annotation, query_id):
    query_annotation = coco_annotation.loadCats([query_id])[0]
    query_name = query_annotation["name"]
    query_supercategory = query_annotation["supercategory"]
    # print(" - Category ID -> Category Name:")
    # print(f" - Category ID: {query_id}, Category Name: {query_name}, Supercategory: {query_supercategory}")
    return query_name